In [1]:
from numpy import *

In [ ]:
def lodaDataSet():
    dataMat = []
    labelMat = []
    fr = open('testSet.txt')
    for line in fr.readlnes():
        lineArr = line.strip().split()
        dataMat.append([1.0, float(lineArr[0]), float(lineArr[1])])
        labelMat.append(int(lineArr[2]))
    return dataMat,labelMat

def sigmoid(inX):
    return 1.0/(1+exp(-inX))

def gradAscent(dataMatIn, classLabels):
    # 将数组转化为numpy矩阵
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    m,n = shape(dataMatrix)
    # alpha 是向目标移动的步长
    alpha = 0.001
    # 迭代次数
    maxCycles = 500
    weights = ones((n, 1))
    for k in range(maxCycles):
        h = sigmoid(dataMatrix*weights)
        error = (labelMat - h)
        # w:=w + alpha × gradient更新回归系数的向量 梯度上升是‘+’，梯度下降是‘-’
        # dataMatrix.transpose()*error 的数学推导还不清楚
        weights = weights + alpha * dataMatrix.transpose()*error
    return weights